# RDFLib Tutorial 

- 작성자: 박하람
- 작성일시: 2022. 09. 05

In [105]:
import numpy as np
import pandas as pd 
from tqdm import tqdm

from rdflib import Namespace, Literal, URIRef
from rdflib.graph import Graph, ConjunctiveGraph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

### 1. 데이터 불러오기

In [106]:
data = pd.read_csv("data/개방목록현황_8월.csv", encoding="utf-8", dtype=str)
data = data.replace({np.nan : None})
data.head()

,목록키,목록유형,목록명,목록설명,조회수,분류체계,기관코드,기관명,국가중점여부,표준데이터여부,목록 등록일,목록 수정일,목록 URL
0,15106236,API,중소벤처기업부_이노비즈확인서,"이노비즈란 ""중소기업기본법"" 제2조제1항에 의한 중소기업으로 ""중소기업 기술혁신 촉...",1,산업고용,1421000,중소벤처기업부,N,N,2022-08-31,2022-08-31,https://www.data.go.kr/data/15106236/openapi.do
1,15106237,API,중소벤처기업부_공고목록정보,"중소벤처기업부 산하/유관기관으로부터 수집하는 공고정보을 제공해 주는 서비스이며, 공...",1,산업고용,1421000,중소벤처기업부,N,N,2022-08-31,2022-08-31,https://www.data.go.kr/data/15106237/openapi.do
2,15106301,API,한국가스공사_충전소 운영정보 목록조회,수소유통전담기관에서 조사한 현재 운영중인 수소충전소 리스트 및 관리번호 목록을 제공...,7,산업고용,B551210,한국가스공사,N,N,2022-08-31,2022-08-31,https://www.data.go.kr/data/15106301/openapi.do
3,15106265,FILE,국민건강보험공단_수가코드별 진료현황,"1. 수진기준(한의분류 제외, 약국 제외), 연령(연말기준)<br/>2. 건강보험 ...",2,보건의료,B550928,국민건강보험공단,N,N,2022-08-31,2022-08-31,https://www.data.go.kr/data/15106265/fileData.do
4,15106270,FILE,국민건강보험공단_해당 수가코드로 인한 입내원일수,"1. 수진기준(한의분류 제외, 약국 제외), 연령(연말기준)<br/>2. 건강보험 ...",2,보건의료,B550928,국민건강보험공단,N,N,2022-08-31,2022-08-31,https://www.data.go.kr/data/15106270/fileData.do


### 2. 데이터 파악하기

In [107]:
print(f"데이터의 행과 열은 {data.shape}입니다.")
print(f"데이터의 컬럼명은 {data.columns}입니다.")

데이터의 행과 열은 (63468, 13)입니다.
데이터의 컬럼명은 Index(['목록키', '목록유형', '목록명', '목록설명', '조회수', '분류체계', '기관코드', '기관명', '국가중점여부',
       '표준데이터여부', '목록 등록일', '목록 수정일', '목록 URL'],
      dtype='object')입니다.


In [108]:
print(f"고유한 목록키의 개수는 {len(data['목록키'].unique())}개입니다.")

고유한 목록키의 개수는 63468개입니다.


In [109]:
data.groupby(["국가중점여부"])['목록키'].count()

국가중점여부
N    60621
Y     2847
Name: 목록키, dtype: int64

In [110]:
data.groupby(["표준데이터여부"])["목록키"].count()

표준데이터여부
N    63321
Y      147
Name: 목록키, dtype: int64

In [111]:
data = data.drop(["목록유형", "조회수", "분류체계"], axis=1)

In [112]:
data[data.columns] = data.apply(lambda x: x.str.strip())

### 3. 데이터 변환하기

#### 변환을 위한 함수 

In [113]:
# namespace 
koor_def = "http://vocab.datahub.kr/def/organization/"
koor_id = "http://data.datahub.kr/id/organization/"
KOOR = Namespace(koor_def)

dcat_id = "http://data.datahub.kr/id/dcat/"

# function (convert data to rdf)
def cell(store, s, p, df_col, datatype = None, lang = None):
    if df_col != None:
        store.add((s, p, Literal(df_col, datatype=datatype, lang = lang)))
        
def uri(store, s, p, df_col, objClass = None, objURI = None) :
    if df_col != None :
        obj = URIRef(objURI + df_col) 
        store.add((s, p, obj))
        if objClass != None :
            store.add((obj, RDF.type, objClass))

In [114]:
%%time

# generate Graph()
g = Graph()
g.bind("koor", KOOR)
g.bind("dcat", DCAT)
g.bind("dct", DCTERMS)

for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
    # base id 
    cat_core_uri = URIRef(dcat_id + "national-cord-data-catalog")
    cat_stan_uri = URIRef(dcat_id + "standard-data-catalog")
    ds_uri = URIRef(dcat_id + "ds-" +row["목록키"])
    dist_uri = URIRef(dcat_id + "dt-" + row["목록키"])
    orga_uri = URIRef(koor_id + row["기관코드"])

    # define uri type
    g.add((cat_core_uri, RDF.type, DCAT.Catalog))
    g.add((cat_stan_uri, RDF.type, DCAT.Catalog))
    g.add((ds_uri, RDF.type, DCAT.Dataset))
    g.add((dist_uri, RDF.type, DCAT.Distribution))
    g.add((ds_uri, DCAT.distribution, dist_uri))

    # dataset in catalog
    if row["국가중점여부"] == "Y":
        g.add((cat_core_uri, DCAT.dataset, ds_uri))
    
    if row["표준데이터여부"] == "Y":
        g.add((cat_stan_uri, DCAT.dataset, ds_uri))

    # catalog metadata
    cell(g, cat_core_uri, DCTERMS.title, "국가중점데이터 목록")
    cell(g, cat_stan_uri, DCTERMS.title, "표준데이터 목록")

    # dataset metadata
    cell(g, ds_uri, DCTERMS.title, row["목록명"], lang="ko")
    cell(g, ds_uri, DCTERMS.description, row["목록설명"], lang="ko")
    uri(g, ds_uri, DCTERMS.publisher, row["기관코드"], objClass=KOOR.Organization, objURI=koor_id)
    cell(g, orga_uri, RDFS.label, row["기관명"], lang="ko")
    cell(g, ds_uri, DCTERMS.issued, row["목록 등록일"], datatype=XSD.date)
    cell(g, ds_uri, DCTERMS.modified, row["목록 수정일"], datatype=XSD.date)
    cell(g, dist_uri, DCAT.accessURL, row["목록 URL"], datatype=XSD.anyURI)

# the number of triples
print(f"총 {len(g)} 개의 트리플이 있습니다.")

# save as ttl
g.serialize(destination=f"data/data-go-kr-metadata-dcat.ttl", format="ttl")

100%|██████████| 63468/63468 [00:16<00:00, 3811.19it/s]


총 576198 개의 트리플이 있습니다.
CPU times: user 41.1 s, sys: 576 ms, total: 41.6 s
Wall time: 41.8 s


### 4. SPARQL 질의하기

In [115]:
# 데이터세트 개수 구하기
query = """
SELECT (COUNT(DISTINCT ?dataset) AS ?dataset_count)
WHERE { 
    ?dataset a dcat:Dataset .
}
"""
result = g.query(query)
for row in result:
    print(f"고유한 데이터세트의 개수는 {row.dataset_count}개입니다.")

고유한 데이터세트의 개수는 63468개입니다.


In [116]:
# 표준데이터 카탈로그와 국가중점데이터 카탈로그에 속한 데이터세트 개수 구하기
query = """
SELECT ?title (COUNT(DISTINCT ?dataset) AS ?dataset_count)
WHERE { 
    ?catalog a dcat:Catalog ;
        dcat:dataset ?dataset ;
        dct:title ?title .
} GROUP BY ?catalog
"""
result = g.query(query)
for row in result:
    print(f"{row.title}의 데이터세트 개수는 {row.dataset_count}개입니다.")

표준데이터 목록의 데이터세트 개수는 147개입니다.
국가중점데이터 목록의 데이터세트 개수는 2847개입니다.


In [118]:
# 데이터세트의 메타데이터 구하기
query = """
SELECT DISTINCT ?dataset ?title ?orgName ?issued ?modified ?accessURL
WHERE { 
    ?dataset a dcat:Dataset ;
        dct:title ?title ;
        dct:publisher ?orgURI ;
        dct:issued ?issued ;
        dct:modified ?modified ;
        dcat:distribution ?distribution .
    ?orgURI rdfs:label ?orgName .
    ?distribution dcat:accessURL ?accessURL .
} LIMIT 10
"""
result = g.query(query)
for row in result:
    print(row.dataset, row.title, row.orgName, row.issued, row.modified, row.accessURL)

http://data.datahub.kr/id/dcat/ds-15087138 한국법제연구원_세계법령정보사이트DB 한국법제연구원 2021-09-02 2021-09-03 https://www.data.go.kr/data/15087138/fileData.do
http://data.datahub.kr/id/dcat/ds-15033949 한국남부발전(주)_설계기술용역 시공도급계약 현황(삼척) 한국남부발전(주) 2021-03-19 2022-03-07 https://www.data.go.kr/data/15033949/fileData.do
http://data.datahub.kr/id/dcat/ds-15086818 대구광역시_(비정형데이터)2021년 대구시 화보집4 대구광역시 2021-08-30 2021-08-30 https://www.data.go.kr/data/15086818/fileData.do
http://data.datahub.kr/id/dcat/ds-15064648 건강보험심사평가원_보건의료빅데이터개방시스템_의료급여진료통계 건강보험심사평가원 2020-09-15 2021-09-22 https://www.data.go.kr/data/15064648/fileData.do
http://data.datahub.kr/id/dcat/ds-15090954 강원도 원주시_원주시청홈페이지 관광포털메뉴 강원도 원주시 2021-09-29 2021-09-29 https://www.data.go.kr/data/15090954/fileData.do
http://data.datahub.kr/id/dcat/ds-15039951 인천광역시 서구_식품접객업소 인천광역시 서구 2019-09-26 2021-09-09 https://www.data.go.kr/data/15039951/fileData.do
http://data.datahub.kr/id/dcat/ds-15099429 경상남도 의령군_약국현황 경상남도 의령군 2022-03-18 2022-03-18 https://www.data.go.kr/d

In [120]:
# 데이터세트를 제공하는 상위 10개 기관 구하기
query = """
SELECT ?orgName (COUNT(DISTINCT ?dataset) AS ?dataset_count)
WHERE { 
    ?dataset a dcat:Dataset ;
        dct:publisher ?org .
    ?org rdfs:label ?orgName .
} GROUP BY ?org
ORDER BY DESC(?dataset_count)
LIMIT 10
"""
result = g.query(query)
for row in result:
    print(f"{row.orgName}의 데이터세트 개수는 {row.dataset_count}개입니다.")

서울특별시의 데이터세트 개수는 2460개입니다.
제주특별자치도의 데이터세트 개수는 1289개입니다.
동북아역사재단의 데이터세트 개수는 1028개입니다.
경기도의 데이터세트 개수는 1015개입니다.
대전광역시의 데이터세트 개수는 991개입니다.
국토교통부의 데이터세트 개수는 955개입니다.
인천광역시의 데이터세트 개수는 780개입니다.
행정안전부의 데이터세트 개수는 779개입니다.
경상남도의 데이터세트 개수는 758개입니다.
국가철도공단의 데이터세트 개수는 749개입니다.
